In [9]:
import numpy as np
import matplotlib.pyplot as plt
import time
from torch import nn

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
device

device(type='cuda')

In [10]:
def calculate_accuracy(dim_outputs, y):
    '''
    For now we will calculate accuracy by model guessing the right amount of dimensions produced
    '''

    accuracies = []

    # y, dim_outputs: list len=batch_size of tensors with shape (dim_count, dim_features)
    for batch_no in range(len(y)):
        actual_dim_number = y[batch_no].shape[0]
        predicted_dim_number = dim_outputs[batch_no].shape[0]
        acc = 0
        if actual_dim_number == 0 and predicted_dim_number == 0:
            acc = 1
        else:
            acc =  0
        # print('expected: {0} predicted: {1}'.format(actual_dim_number, predicted_dim_number))
        accuracies.append(acc)
        
    return accuracies

def plot_history(loss_history, train_history, val_history):
    '''
    Plots learning history in jupyter
    '''
    plt.ylabel('Accuracy @ epoch')
    
    train, = plt.plot(train_history)
    train.set_label("train")

    validation, = plt.plot(val_history)
    validation.set_label("validation")

    loss, = plt.plot(loss_history)
    loss.set_label("loss")

    plt.legend()
    plt.show()
    
def train_model(model, train_loader, val_loader, loss, optimizer, epochs):
    '''
    trains model and outputs loss, train and validation history
    '''

    start = time.time()
    
    loss_history   = []
    train_history  = []
    val_history    = []
    train_accuracy = 0.0

    
    for epoch in range(epochs):
        model.train()
        
        loss_accumulated = 0
        
        for i, (x, y) in enumerate(train_loader):
            
            model.zero_grad()
            
            outs_num = model(x.to(device))
            
            loss_value = loss(outs_num, y)
            
            loss_value.backward()
            optimizer.step()
            
            loss_accumulated += loss_value
            
            train_accuracy = np.mean(calculate_accuracy(decoded, y))

            print('[{0}-{1} @ {2:.1f} sec] Log10 Loss: {3:2f} Train err: {4:2.1f}%'. format(
                epoch,
                i,
                time.time() - start,
                np.log10(float(loss_value)),
                (1 - train_accuracy) * 100
            ))

        train_history.append(train_accuracy)
        loss_history.append(float(loss_accumulated))

        # validation
        model.eval()

        val_accuracies = []
        for _, (x, y) in enumerate(val_loader):
            predictions = model(x)

            val_acc = np.mean(calculate_accuracy(predictions, y))
            val_accuracies.append(val_acc)
        val_accurcy = np.mean(val_accuracies)
        print('Epoch validation accuracy: {0:2.3f}'.format(val_accurcy))
        val_history.append(val_accurcy)

    return loss_history, train_history, val_history

In [11]:
#  https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817
import torch
from dataset import DwgDataset

dwg_dataset = DwgDataset(pickle_file='test_dataset.pickle', batch_size=batch_size)

train_loader = dwg_dataset.train_loader
val_loader   = dwg_dataset.val_loader
test_loader  = dwg_dataset.test_loader
 
ent_features = dwg_dataset.entities.ent_features
dim_features = dwg_dataset.entities.dim_features

In [12]:
from torch.autograd import Variable
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

model = LinearRegression(2, 1)
model.to(device)


LinearRegression(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [13]:
lr = 1e-3
epochs = 11    
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_history, train_history, val_history = train_model(
    model      =   model, 
    train_loader = train_loader,
    val_loader   = val_loader,
    loss         = loss,
    optimizer  = optimizer,
    epochs       = epochs)

plot_history(loss_history, train_history, val_history)

AttributeError: 'list' object has no attribute 'to'